## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-06-20-50-01 +0000,nypost,Hamas terrorist who bragged about slaughtering...,https://nypost.com/2025/09/06/world-news/hero-...
1,2025-09-06-20-43-18 +0000,nyt,L.G.B.T.Q. Catholics Have Jubilee With Pope’s ...,https://www.nytimes.com/2025/09/06/world/europ...
2,2025-09-06-20-13-51 +0000,nypost,Man shot in neck as bullets fly in front of Br...,https://nypost.com/2025/09/06/us-news/man-shot...
3,2025-09-06-20-09-53 +0000,latimes,Chargers-Chiefs game in Brazil a blueprint for...,https://www.latimes.com/sports/chargers/story/...
4,2025-09-06-20-06-15 +0000,startribune,Memorial set for 10-year-old girl killed in An...,https://www.startribune.com/celebration-of-lif...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2374/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
252,trump,39
430,new,11
180,war,10
268,immigration,10
67,report,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
195,2025-09-05-22-57-00 +0000,wsj,President Trump signed an executive order to r...,https://www.wsj.com/politics/national-security...,83
128,2025-09-06-09-00-00 +0000,wsj,Rep. Marjorie Taylor Greene is walking a tight...,https://www.wsj.com/politics/policy/marjorie-t...,72
174,2025-09-06-00-36-12 +0000,nyt,Trump Says U.S. Military Has ‘Never Fought to ...,https://www.nytimes.com/2025/09/05/us/politics...,72
206,2025-09-05-22-15-00 +0000,wsj,Opinion | Can Trump’s ‘War Department’ Win a War?,https://www.wsj.com/opinion/department-of-war-...,69
123,2025-09-06-09-01-55 +0000,nyt,"George Mason’s President, Gregory Washington, ...",https://www.nytimes.com/2025/09/06/us/george-m...,66


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
195,83,2025-09-05-22-57-00 +0000,wsj,President Trump signed an executive order to r...,https://www.wsj.com/politics/national-security...
55,43,2025-09-06-15-27-41 +0000,nyt,South Koreans Are Swept Up in Immigration Raid...,https://www.nytimes.com/2025/09/05/us/georgia-...
235,40,2025-09-05-20-58-03 +0000,nypost,Top secret SEAL Team 6 killed North Korean civ...,https://nypost.com/2025/09/05/us-news/top-secr...
115,36,2025-09-06-09-46-41 +0000,bbc,'No doubt' Russia will cross Nato border if Uk...,https://www.bbc.com/news/videos/cq65410424qo?a...
186,34,2025-09-05-23-23-22 +0000,nyt,Off-Duty Pilot Who Tried to Shut Off Engines E...,https://www.nytimes.com/2025/09/05/us/alaska-a...
187,28,2025-09-05-23-19-00 +0000,wsj,The IT unemployment rate fell to 4.5% in Augus...,https://www.wsj.com/articles/it-unemployment-f...
26,27,2025-09-06-17-59-05 +0000,bbc,Chris Mason: Reform conference shows party's g...,https://www.bbc.com/news/articles/c62zdpke8kko...
134,26,2025-09-06-07-23-56 +0000,nypost,Joe Kennedy III blasts RFK Jr. after fiery Sen...,https://nypost.com/2025/09/06/us-news/joe-kenn...
217,25,2025-09-05-21-41-53 +0000,nyt,Israel Steps Up Attacks on Gaza City Ahead of ...,https://www.nytimes.com/2025/09/05/world/middl...
36,24,2025-09-06-17-10-04 +0000,nypost,Judge blocks NYC from opening ‘insane’ homeles...,https://nypost.com/2025/09/06/us-news/judge-bl...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
